In [2]:
# !pip install hydrafloods

In [4]:
import hydrafloods as hf
import ee

# initialize GEE
# ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [22]:
start_time = "2019-10-05"
end_time   =  "2019-10-06"
region     = ee.Geometry.Rectangle([104, 11.5, 106, 12.5 ])

In [8]:
s1 = hf.datasets.Sentinel1(region, start_time, end_time)

In [10]:
water_imgs = s1.apply_func(
                            hf.thresholding.edge_otsu,
                            initial_threshold=-14,
                            thresh_no_data=-20,
                            edge_buffer=300
                          )

In [11]:
water_map = ee.Image(water_imgs.collection.mode())

In [13]:
hf.geeutils.export_image(
    water_map,
    region,
    "users/mafmonjaraz/water_map_example",
    scale=30,
)

In [27]:
# !pip install folium
import folium
from folium import plugins
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):

    try:    

        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True,
            style_function=lambda x:vis_params
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [30]:
lon_lat         =  [104, 11.5]
my_map = folium.Map(location= [lon_lat[1], lon_lat[0]], zoom_start=12)
basemaps['Esri Satellite'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)
vis    = {"min":0, "max":5 ,"palette":["ff4545","a7ff7a","1f1e6e"]}
my_map.add_ee_layer(region, {}, 'region')
my_map.add_ee_layer(water_map,vis, 'my_map')

my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

display(my_map)
